# Install and Import Required libraries
- We assume that the free software Google Sheets & Drive can be accessed.
- This was for our convenience, and is not, strictly speaking, necessary. However, our code will need to be modified if other software are used.

In [1]:
# Install necessary libraries if not already installed
!pip install --upgrade pandas==2.2.2 # google-colab 1.0.0 requires pandas==2.2.2
!pip install --upgrade gspread gspread_dataframe

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import gspread
from google.colab import auth
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.auth import default
import itertools
from IPython.display import display, HTML

# Load pre-processed Census Data

## What data are we using?
Last Updated: 04-Oct-2024
- Three datasets downloaded from the website of [Census of India](https://censusindia.gov.in/census.website/data/census-tables):
  - Census table B01: Main workers, Marginal workers and Non-workers (total)
  - Census table B04: Main workers (i.e., workers who have been employed for 6 months or more)
  - Census table B06: Marginal workers (i.e., workers who have been employed for less than 6 months)
- One **important** thing to note is that the B01 table also reports Total Population figure of 1,67,87,941, which should equal the sum of Main Workers, Marginal Workers and Non Workers.
  - This expectation holds if we only use the data in the B01 table
  - However, the total number of Main and Marginal Workers reported in B01 do not match that reported in tables B04 or B06, the latter two reporting higher totals
  - **This causes small discrepancies**. For instance, when we sum up total number of Main Workers from B04, Marginal Workers from B06 and Non Workers from B01, we end up with a total population of 1,67,90,207 (a 0.01% increase)
- Each of these datasets provide counts of workers by District (Code and Name), District Type (Rural/Urban), Age and Gender
  - Each district had a unique name and a corresponding code, which we verified later (see code further below) to be a unique corresponding code
- Additionally, the Main workers and Marginal workers datasets offer counts of workers by Job Type
- Reference: https://www.data.gov.in/catalog/main-workers-marginal-workers-non-workers-and-those-marginal-workers-non-workers

## How to download the data?
Last Updated: 04-Oct-2024
1. Go to https://censusindia.gov.in/census.website/data/census-tables
2. Select the latest Census year (e.g., 2011)
3. Select the Table series “Workers”
4. In the large search bar on the top half of the screen, search for “delhi” or “Delhi”
5. Pick up the appropriate dataset (use judgement) and download it as an Excel spreadsheet

<div align="center">
  <img src="https://drive.google.com/uc?export=view&id=1JYIfxLLqVW34aJfT4klEMBXstJ9mr3zm" alt="My Image">
</div>

## What pre-processing was done?
- Each dataset was formatted to have a single header row of column names (the raw data had multiple column names)
  - This also involved looking up a "key" for job type abbreviations, as presented below the data table in the file downloaded from the Census website
- From the dataset which included Non Workers data (the B01 Census table), only Non Workers data were used as more granular data was available in the other two datasets (B04 and B06) for main and marginal workers
- Finally, the three datasets were merged based on District (Code and Name), District Type (Rural/Urban), Age and Gender

## Code below to load pre-processed data into Python environment

In [3]:
# Authenticate and create the client
auth.authenticate_user()
creds, _ = default()
client = gspread.authorize(creds)

In [4]:
# Open the Google Sheet by URL
sheet_url = "https://docs.google.com/spreadsheets/d/1kBNrbqClu3SDH6mbDtijaMe5iQ-sDWjPc1K2tNAEDIM/edit?gid=261375631#gid=261375631"
sheet = client.open_by_url(sheet_url)

# Select the specific worksheet (tab) to work with
worksheet = sheet.worksheet('Main + Marginal + Non-Workers')

# Load the data into a pandas DataFrame
df = get_as_dataframe(worksheet, dtype={'Age_Group': str, 'District_Code': str})

# Display the first few rows to confirm it loaded correctly
df.head()

,District_Code,Area_Name,Area_Type,Age_Group,Main_Workers_Total,Main_Workers_Males,Main_Workers_Females,Main_Cultivators_Total,Main_Cultivators_Males,Main_Cultivators_Females,...,"Marginal_Education, Human Health and Social Work activities_Females","Marginal_Arts, Entertainment and Recreation, Other Service Activities, Activities of Households as Employers: Undifferentiated Goods and Services, Activities of Extra-Territorial Organisations and Bodies (Household Industries)_Total","Marginal_Arts, Entertainment and Recreation, Other Service Activities, Activities of Households as Employers: Undifferentiated Goods and Services, Activities of Extra-Territorial Organisations and Bodies (Household Industries)_Males","Marginal_Arts, Entertainment and Recreation, Other Service Activities, Activities of Households as Employers: Undifferentiated Goods and Services, Activities of Extra-Territorial Organisations and Bodies (Household Industries)_Females","Marginal_Arts, Entertainment and Recreation, Other Service Activities, Activities of Households as Employers: Undifferentiated Goods and Services, Activities of Extra-Territorial Organisations and Bodies (Non-Household Industries)_Total","Marginal_Arts, Entertainment and Recreation, Other Service Activities, Activities of Households as Employers: Undifferentiated Goods and Services, Activities of Extra-Territorial Organisations and Bodies (Non-Household Industries)_Males","Marginal_Arts, Entertainment and Recreation, Other Service Activities, Activities of Households as Employers: Undifferentiated Goods and Services, Activities of Extra-Territorial Organisations and Bodies (Non-Household Industries)_Females",Non Workers_Total,Non Workers_Males,Non Workers_Females
0,000,State - NCT OF DELHI,Total,Total,5309803.0,4565137.0,744666.0,27759.0,24225.0,3534.0,...,8772.0,4108.0,1886.0,2222.0,59908.0,28564.0,31344.0,11200892.0,4225300.0,6975592.0
1,000,State - NCT OF DELHI,Total,5-9,6255.0,3721.0,2534.0,257.0,155.0,102.0,...,14.0,61.0,23.0,38.0,2807.0,1498.0,1309.0,1526399.0,823934.0,702465.0
2,000,State - NCT OF DELHI,Total,10-14,20394.0,15282.0,5112.0,296.0,200.0,96.0,...,18.0,113.0,52.0,61.0,3873.0,1936.0,1937.0,1621403.0,876188.0,745215.0
3,000,State - NCT OF DELHI,Total,15-19,166072.0,144747.0,21325.0,852.0,656.0,196.0,...,357.0,372.0,226.0,146.0,7617.0,4384.0,3233.0,1476287.0,761988.0,714299.0
4,000,State - NCT OF DELHI,Total,20-24,576739.0,494204.0,82535.0,2577.0,2255.0,322.0,...,1916.0,637.0,352.0,285.0,10872.0,6049.0,4823.0,1136490.0,419129.0,717361.0


# "Melting" the Data
- As can be seen, the counts of workers by Job Type and Gender have been tabled as columns
- For the convenience of not needing to working with a large number of columns (127 in this case), we "melt" down this dataset from a "wide" format (large number of columns) to a "long" format dataset (smaller number of columns, but with more rows)

In [5]:
# Columns that remain fixed
id_vars = list(df.columns[:4]) # ['District_Code', 'Area_Name', 'Area_Type', 'Age_Group']

# Columns to unpivot: counts of citzens by job type and gender
value_vars = list(df.columns[4:]) # all columns after column 4, i.e., 'Age_Group'

# Melt the dataset - this converts the "wide" format data to "long" format
long_df = pd.melt(df, id_vars=id_vars, value_vars=value_vars,
                  var_name='Job_Type_Gender', value_name='Count')

# Split 'Job_Type_Gender' into separate 'Job_Type' and 'Gender' columns
def split_job_type_gender(col):
    # Define gender terms to identify
    gender_terms = ['Males', 'Females', 'Total']

    # Search for gender terms in the column name
    for gender in gender_terms:
        if gender in col:
            # Split at the gender term
            job_type = col.replace('_' + gender, '')  # Remove gender part
            return job_type, gender
long_df['Job_Type'], long_df['Gender'] = \
  zip(*long_df['Job_Type_Gender'].apply(split_job_type_gender))

# Drop the original 'Job_Type_Gender' column
long_df = long_df.drop(columns=['Job_Type_Gender'])

# Display the first few rows of the long format dataframe
long_df.head()

,District_Code,Area_Name,Area_Type,Age_Group,Count,Job_Type,Gender
0,000,State - NCT OF DELHI,Total,Total,5309803.0,Main_Workers,Total
1,000,State - NCT OF DELHI,Total,5-9,6255.0,Main_Workers,Total
2,000,State - NCT OF DELHI,Total,10-14,20394.0,Main_Workers,Total
3,000,State - NCT OF DELHI,Total,15-19,166072.0,Main_Workers,Total
4,000,State - NCT OF DELHI,Total,20-24,576739.0,Main_Workers,Total


## What does our Long Format dataset look like?
Our “long format” data table has the following columns:

- District_Code: categorical, with 10 levels, including "0", which represents totals for the whole state of Delhi
- Area_Name: categorical, with 10 levels, including "State - NCT OF DELHI", which represents totals for the whole state
  - Note that Area_Name maps 1:1 with District_Code; this was verified separately!
- Area_Type: categorical, with 3 levels, including "Total"
- Age_Group: categorical, with 14 levels, including "Total"
- Job_Type: categorical, with 20 levels, including "Main Workers", which represents totals over all Job Types
- Gender: categorical, with 3 levels, "Males", "Females" and "Total", which was the sum total over the two other levels
- Count: numerical data representing the count of workers in the specified categories


# Handling a formatting issue
- The Age Group column has categories such as "5-9", which may be misinterpreted by Google Sheets (a tool we may need further downstream), we replace the "-" with " to ", converting "5-9" for example, to "5 to 9"

In [6]:
# Ensure that age groups, for e.g., "5-9" is not misinterpreted by Google Sheets as a Date (9th of May)
long_df['Age_Group'] = long_df['Age_Group'].apply(lambda x: x.replace("-", " to "))
long_df.head()

,District_Code,Area_Name,Area_Type,Age_Group,Count,Job_Type,Gender
0,000,State - NCT OF DELHI,Total,Total,5309803.0,Main_Workers,Total
1,000,State - NCT OF DELHI,Total,5 to 9,6255.0,Main_Workers,Total
2,000,State - NCT OF DELHI,Total,10 to 14,20394.0,Main_Workers,Total
3,000,State - NCT OF DELHI,Total,15 to 19,166072.0,Main_Workers,Total
4,000,State - NCT OF DELHI,Total,20 to 24,576739.0,Main_Workers,Total


# Validate the Data

In [7]:
# Check that District_Code and Area_Name have 1:1 mapping
long_df[['District_Code', 'Area_Name']].drop_duplicates().reset_index()

,index,District_Code,Area_Name
0,0,000,State - NCT OF DELHI
1,42,090,District - North West
2,84,091,District - North
3,126,092,District - North East
4,168,093,District - East
5,210,094,District - New Delhi
6,252,095,District - Central
7,294,096,District - West
8,336,097,District - South West
9,378,098,District - South


## Scope - i.e., what we can validate?
- Now that we have validated the 1:1 unique mapping between District Code and Area Name (District Name), what else can we validate?

- The only thing we can further validate (without referring to other data sources) is the consistency of the "Total" values in the data table.

- Specifically:

  - We can validate that for each categorical variable, the "Total" level consistently represents the sum of the Count values across all other levels (i.e., all the non-total levels, or "base levels") within that same variable.

  - For example, we can check if the Count value for Area_Name = Total equals the sum of the Count values for all base levels of Area_Name (i.e., Rural and Urban). We could call this “**Univariate Validation**”.

  - We can also validate that "Total" values are consistent across different categorical variables when aggregating Count values. This means that we can check if the Count value associated with a combination of "Total" levels across multiple variables (e.g., Area_Name = Total, Area_Type = Total, etc.) equals the sum of the Count values across all possible combinations of base levels within those variables.

  - We could call this “**Multivariate Validation**”. It is easy to see that if the data are valid according to the Multivariate Validation check, they should be valid according to the Univariate Validation check as well.

### Partition data out:
- Full Dataset = Main Workers Data + Marginal Workers Data + Non Workers Data
- We want to check for consistencies in counts of workers within each partition

In [8]:
df_for_validation_main = long_df.loc[long_df['Job_Type'].str.startswith("Main_"), :]
df_for_validation_marginal = long_df.loc[long_df['Job_Type'].str.startswith("Marginal_"), :]
df_for_validation_non = long_df.loc[long_df['Job_Type'] == "Non Workers", :]

### Reusable Functions to conduct our Multivariate Validation of Worker Counts

In [9]:
def separate_base_and_total_levels_data(df, total_level_labels):
    """
    Separate base-level and total-level data based on total labels.

    Parameters:
    df                : DataFrame containing both base and total level data
    total_level_labels: dict where keys = categorical columns,
                        and values = corresponding total level label

    Returns:
    base_levels_data : DataFrame containing only base-level rows
    total_levels_data: DataFrame containing only total-level rows
    """

    # Create a boolean DataFrame to check if any column contains total level labels
    total_mask = pd.DataFrame({col: df[col] == total_level_labels[col] \
                               for col in total_level_labels.keys()})

    # Rows that have any total level labels (True in any column)
    total_levels_data_mask = total_mask.any(axis=1)

    # Separate base-level and total-level data
    base_levels_data = df[~total_levels_data_mask].copy()
    total_levels_data = df[total_levels_data_mask].copy()

    return base_levels_data, total_levels_data

In [10]:
def compare_totals(row, base_levels_data, total_level_labels, numerical_var_label):
    """
    Compare totals between total-level row and corresponding base-level data.

    Parameters:
    row               : one row of total_levels_data
    base_levels_data  : DataFrame of base level data
    total_level_labels: dict where keys = categorical columns,
                        and values = corresponding total level label
    numerical_var_label: column name of the numerical variable to sum

    Returns:
    A pandas Series containing computed total, reported total, and the difference,
    if mismatched totals are found.
    """

    # Create a filtering condition excluding columns that match total labels
    query_str = ' & '.join(
        f"{col} == '{row[col]}'"
        for col in total_level_labels
        if row[col] != total_level_labels[col]
    )

    # Apply the query to filter the base levels data
    if query_str:
        filtered_data = base_levels_data.query(query_str)
    else:
        filtered_data = base_levels_data

    # Sum the numerical variable for the filtered data
    computed_total = filtered_data[numerical_var_label].sum()

    # Get the reported total from the row
    reported_total = row[numerical_var_label]

    # Return a Series if there's a mismatch
    if computed_total != reported_total:
        return pd.Series({
            'Computed Total': computed_total,
            'Reported Total': reported_total,
            'Difference': reported_total - computed_total
        })


In [11]:
def multivariate_validation(df, total_level_labels,
                            numerical_var_label='Count'):

  base_levels_data, total_levels_data = \
    separate_base_and_total_levels_data(df, total_level_labels)

  # Apply the function to total_data
  mismatches = total_levels_data.apply(
      lambda row: compare_totals(row, base_levels_data,
                                  total_level_labels, numerical_var_label),
      axis=1
  ).dropna()

  # Display mismatches
  if not mismatches.empty:
      print("Mismatched Totals Found:")
      result = total_levels_data.loc[mismatches.index,
                                     total_level_labels.keys()].join(mismatches)
      display(HTML(result.reset_index(drop=True).to_html(index=False)))
      return result
  else:
      print("No mismatches found.")
      return None

In [12]:
# Validate Main Workers part of the dataset
total_level_labels = {
    'Area_Name': 'State - NCT OF DELHI ', # note the trailing space!
    'Area_Type': 'Total',
    'Age_Group': 'Total',
    'Job_Type': 'Main_Workers',
    'Gender': 'Total'
}
result = multivariate_validation(df_for_validation_main, total_level_labels)

No mismatches found.


In [13]:
# Validate Marginal Workers part of the dataset
total_level_labels = {
    'Area_Name': 'State - NCT OF DELHI ', # note the trailing space!
    'Area_Type': 'Total',
    'Age_Group': 'Total',
    'Job_Type': 'Marginal_Workers',
    'Gender': 'Total'
}
result = multivariate_validation(df_for_validation_marginal, total_level_labels)

No mismatches found.


In [14]:
# Validate Non Workers part of the dataset
# Note that the Non Workers has no Job Type other than "Non_Workers"
total_level_labels = {
    'District_Code': '000',
    'Area_Name': 'State - NCT OF DELHI ', # note the trailing space!
    'Area_Type': 'Total',
    'Age_Group': 'Total',
    'Gender': 'Total'
}
result = multivariate_validation(df_for_validation_non, total_level_labels)

Mismatched Totals Found:


District_Code,Area_Name,Area_Type,Age_Group,Gender,Computed Total,Reported Total,Difference
000,State - NCT OF DELHI,Total,Total,Total,9819692.0,11200892.0,1381200.0
000,State - NCT OF DELHI,Rural,Total,Total,249690.0,288815.0,39125.0
000,State - NCT OF DELHI,Urban,Total,Total,9570002.0,10912077.0,1342075.0
090,District - North West,Total,Total,Total,2160580.0,2467994.0,307414.0
090,District - North West,Rural,Total,Total,126587.0,146458.0,19871.0
090,District - North West,Urban,Total,Total,2033993.0,2321536.0,287543.0
091,District - North,Total,Total,Total,520887.0,591532.0,70645.0
091,District - North,Rural,Total,Total,10881.0,12468.0,1587.0
091,District - North,Urban,Total,Total,510006.0,579064.0,69058.0
092,District - North East,Total,Total,Total,1373175.0,1580238.0,207063.0


# We found an issue with the Non Workers part of our data!
- Inspecting the results of the validation procedure for the same, we see that Age_Group alone is 'Total' in all mismatched total level rows.
- This points to there being a problem with the Age_Group column.
- Upon further inspection of the raw data, it became clear that whereas the Total appears to include counts of non-workers of age 0-4, without a corresponding base level
- To fix this, missing data for age group 0-4 was added, based on the results that were obtained through the validation procedure
- In the printed (and returned) result of the validation procedure, the column "Difference" is the count of non workers corresponding to age group 0-4
- So, just formatting and appending the returned result to `long_df` would do the job

In [15]:
excluded_columns = ['Age_Group', 'Computed Total', 'Reported Total']
selected_df = result.drop(columns=excluded_columns)
selected_df['Job_Type'] = 'Non Workers'
selected_df['Age_Group'] = '0-4'
selected_df = selected_df.rename(columns={'Difference': 'Count'})
selected_df = selected_df[long_df.columns] # reorder exactly like long_df

long_df = pd.concat([long_df, selected_df], ignore_index = True) # add selected data to long_df

In [16]:
# revalidate to ensure that the issue is solved
df_for_validation_non = long_df.loc[long_df['Job_Type'] == "Non Workers", :]
result = multivariate_validation(df_for_validation_non, total_level_labels)

No mismatches found.


This verifies that our data validation issue is now solved!

# Keep only those rows corresponding to base levels of categorical variables

In [17]:
# Get Base Levels Data for Main Workers
total_level_labels = {
    'Area_Name': 'State - NCT OF DELHI ', # note the trailing space!
    'Area_Type': 'Total',
    'Age_Group': 'Total',
    'Job_Type': 'Main_Workers',
    'Gender': 'Total'
}
base_levels_main, _ = \
  separate_base_and_total_levels_data(df_for_validation_main,
                                      total_level_labels)

# Get Base Levels Data for Marginal Workers
total_level_labels = {
    'Area_Name': 'State - NCT OF DELHI ', # note the trailing space!
    'Area_Type': 'Total',
    'Age_Group': 'Total',
    'Job_Type': 'Marginal_Workers',
    'Gender': 'Total'
}
base_levels_marginal, _ = \
  separate_base_and_total_levels_data(df_for_validation_marginal,
                                      total_level_labels)

# Get Base Levels Data for Non Workers
total_level_labels = {
    'Area_Name': 'State - NCT OF DELHI ', # note the trailing space!
    'Area_Type': 'Total',
    'Age_Group': 'Total',
    'Gender': 'Total'
}
base_levels_non, _ = \
  separate_base_and_total_levels_data(df_for_validation_non,
                                      total_level_labels)

# Merge all the Base Levels Datasets
base_levels_df = pd.concat(
    [base_levels_main, base_levels_marginal, base_levels_non],
    ignore_index = True)

In [18]:
base_levels_df

,District_Code,Area_Name,Area_Type,Age_Group,Count,Job_Type,Gender
0,090,District - North West,Rural,5 to 9,17.0,Main_Cultivators,Males
1,090,District - North West,Rural,10 to 14,22.0,Main_Cultivators,Males
2,090,District - North West,Rural,15 to 19,78.0,Main_Cultivators,Males
3,090,District - North West,Rural,20 to 24,344.0,Main_Cultivators,Males
4,090,District - North West,Rural,25 to 29,565.0,Main_Cultivators,Males
...,...,...,...,...,...,...,...
18279,096,District - West,Urban,0-4,92628.0,Non Workers,Females
18280,097,District - South West,Rural,0-4,5879.0,Non Workers,Females
18281,097,District - South West,Urban,0-4,79383.0,Non Workers,Females
18282,098,District - South,Rural,0-4,588.0,Non Workers,Females


# Aggregate Main and Marginal Worker Counts by Job Type
- Since we are not proposing different actions for Main and Marginal Workers, we can simplify our analysis by aggregating these counts

In [19]:
# Step 1: Extract basic job type into a new column (remove Main_ and Marginal_ prefixes)
base_levels_df['Basic_Job_Type'] = \
  base_levels_df['Job_Type'].str.replace(r"^(Main_|Marginal_)", "", regex=True)

# Step 2: Initialize Main_Count and Marginal_Count columns based on the original Job_Type
base_levels_df['Main_Count'] = \
  np.where(base_levels_df['Job_Type'].str.startswith("Main_"),
           base_levels_df['Count'], 0)
base_levels_df['Marginal_Count'] = \
  np.where(base_levels_df['Job_Type'].str.startswith("Marginal_"),
           base_levels_df['Count'], 0)

# Step 3: For Non_Workers, set both Main_Count and Marginal_Count to 0
base_levels_df.loc[base_levels_df['Job_Type'] == 'Non_Workers',
 ['Main_Count', 'Marginal_Count']] = 0

# Step 4: Set Count = Main_Count + Marginal_Count, except for Non_Workers
# For Non_Workers, retain the original Count value
base_levels_df['Total_Count'] = np.where(
    base_levels_df['Job_Type'] != 'Non_Workers',
    base_levels_df['Main_Count'] + base_levels_df['Marginal_Count'],
    base_levels_df['Count']
)

# Step 5: Group by 'District_Code', 'Area_Name', 'Area_Type', 'Age_Group', 'Gender', 'Basic_Job_Type'
# and sum up Main_Count, Marginal_Count, and Count
aggregated_df = base_levels_df.groupby(
    ['District_Code', 'Area_Name', 'Area_Type',
     'Age_Group', 'Gender', 'Basic_Job_Type']
).agg(
    Main_Count=('Main_Count', 'sum'),
    Marginal_Count=('Marginal_Count', 'sum'),
    Total_Count=('Count', 'sum')
).reset_index()

# Check the aggregated DataFrame
aggregated_df

,District_Code,Area_Name,Area_Type,Age_Group,Gender,Basic_Job_Type,Main_Count,Marginal_Count,Total_Count
0,090,District - North West,Rural,0-4,Females,Non Workers,0.0,0.0,8989.0
1,090,District - North West,Rural,0-4,Males,Non Workers,0.0,0.0,10882.0
2,090,District - North West,Rural,10 to 14,Females,Accommodation and Food Service,0.0,0.0,0.0
3,090,District - North West,Rural,10 to 14,Females,"Administrative and Support Service, Public Adm...",0.0,0.0,0.0
4,090,District - North West,Rural,10 to 14,Females,Agricultural Labour,9.0,2.0,11.0
...,...,...,...,...,...,...,...,...,...
9387,098,District - South,Urban,Age not stated,Males,Non Workers,0.0,0.0,953.0
9388,098,District - South,Urban,Age not stated,Males,"Plantation, Livestock, Forestry, Fishing, Hunt...",3.0,0.0,3.0
9389,098,District - South,Urban,Age not stated,Males,Transportation and Storage,46.0,3.0,49.0
9390,098,District - South,Urban,Age not stated,Males,Wholesale and Retail Trade (incl. Repair of mo...,0.0,0.0,0.0


# Abbreviate Job Types meaningfully
- As can be seen, the raw Job Types (as fetched from the Census dataset's key) can be too long
- We therefore map each Job Type to an abbreviated Job Type; see mapping in the code cell below

In [20]:
# Create the mapping as a dictionary
job_type_mapping = {
  'Workers': 'Workers',
  'Cultivators': 'Cultivators',
  'Agricultural Labour': 'Food & Cash Crop Labour',
  'Plantation, Livestock, Forestry, Fishing, Hunting and allied activities': 'Other Food Production',
  'Mining and Quarrying': 'Mining and Quarrying',
  'Manufacturing (Household Industry)': 'Manufacturing (HHI)',
  'Manufacturing (Non-Household Industries)': 'Manufacturing (Non-HHI)',
  'Electricity, Gas, Steam, Air Conditioning Supply, Water Supply, Sewerage, Waste Management and Remediation activities': 'Utilities',
  'Construction': 'Construction',
  'Wholesale and Retail Trade (incl. Repair of motor vehicles and motor cycles) (Household Industries)': 'Trade (HHI)',
  'Wholesale and Retail Trade (incl. Repair of motor vehicles and motor cycles) (Non-Household Industries)': 'Trade (Non-HHI)',
  'Transportation and Storage': 'Transportation & Storage',
  'Accommodation and Food Service': 'Hospitality & Dining',
  'Information and Communication (Household Industries)': 'IT & Communication (HHI)',
  'Information and Communication (Non-Household Industries)': 'IT & Communication (Non-HHI)',
  'Finance, Insurance, Real Estate activities, Professional, Scientific and Technical activities': 'Finance, Science & Other Tech',
  'Administrative and Support Service, Public Administration and Defence, Compulsory Social Security': 'Administration',
  'Education, Human Health and Social Work activities': 'Education, Health & Social Work',
  'Arts, Entertainment and Recreation, Other Service Activities, Activities of Households as Employers: Undifferentiated Goods and Services, Activities of Extra-Territorial Organisations and Bodies (Household Industries)': 'Arts and Other Sectors (HHI)',
  'Arts, Entertainment and Recreation, Other Service Activities, Activities of Households as Employers: Undifferentiated Goods and Services, Activities of Extra-Territorial Organisations and Bodies (Non-Household Industries)': 'Arts and Other Sectors (Non-HHI)',
  'Non Workers': 'Non Workers'
}

# Add an "Abbreviated Job Type" column using the map function
aggregated_df['Abbreviated_Job_Type'] = aggregated_df['Basic_Job_Type'].map(job_type_mapping)

# Display the DataFrame with the new column
aggregated_df.head()


,District_Code,Area_Name,Area_Type,Age_Group,Gender,Basic_Job_Type,Main_Count,Marginal_Count,Total_Count,Abbreviated_Job_Type
0,090,District - North West,Rural,0-4,Females,Non Workers,0.0,0.0,8989.0,Non Workers
1,090,District - North West,Rural,0-4,Males,Non Workers,0.0,0.0,10882.0,Non Workers
2,090,District - North West,Rural,10 to 14,Females,Accommodation and Food Service,0.0,0.0,0.0,Hospitality & Dining
3,090,District - North West,Rural,10 to 14,Females,"Administrative and Support Service, Public Adm...",0.0,0.0,0.0,Administration
4,090,District - North West,Rural,10 to 14,Females,Agricultural Labour,9.0,2.0,11.0,Food & Cash Crop Labour


# Formatting: Reorder columns so that count columns appear at the end

In [21]:
# Reordered Column Names
reordered_columns = [aggregated_df.columns[i] for i in [0, 1, 2, 3, 4, 5, 9, 6, 7, 8]]

# Reorder the DataFrame columns
aggregated_df = aggregated_df[reordered_columns]

# Display the DataFrame to verify column order
aggregated_df.head()

,District_Code,Area_Name,Area_Type,Age_Group,Gender,Basic_Job_Type,Abbreviated_Job_Type,Main_Count,Marginal_Count,Total_Count
0,090,District - North West,Rural,0-4,Females,Non Workers,Non Workers,0.0,0.0,8989.0
1,090,District - North West,Rural,0-4,Males,Non Workers,Non Workers,0.0,0.0,10882.0
2,090,District - North West,Rural,10 to 14,Females,Accommodation and Food Service,Hospitality & Dining,0.0,0.0,0.0
3,090,District - North West,Rural,10 to 14,Females,"Administrative and Support Service, Public Adm...",Administration,0.0,0.0,0.0
4,090,District - North West,Rural,10 to 14,Females,Agricultural Labour,Food & Cash Crop Labour,9.0,2.0,11.0


# Identifying Job-Type Agnostic Vulnerable Groups
- Workers below the age of 15 (children) and those above the age of 69 (seniors) are widely considered to be vulnerable (see newsletter for references)
  - Note that workers above the age of 64 are themselves widely considered vulnerable, but since the Census datasets do not provide counts of workers in the age group 60-64 but rather only for 60-69, we use age 69 as a cut-off for our analysis (but not our recommended action plan)
- Pregnant (female) workers are also widely considered vulnerable
  - However, the Census datasets do not allow us to identify pregnant females
  - Instead, we are **estimating** the count of pregnant female workers using Age Specific Fertility Rate (ASFR) data from the Sample Registration System (SRS) statistical report of 2011 https://censusindia.gov.in/nada/index.php/catalog/34790, produced by the Office of the Registrar General & Census Commissioner, India (ORGI)
    - Since the census data of workers does not split the age group 40-49 any further, the ASFR for this age group is averaged as (AFSR for age group 40-44 + AFSR for age group 45-49)/2
  - We estimate number of pregnant women as count of female workers in a given age group times ASFR divided by 1000 times 9/12, where the factor 9/12 accounts for the fact that a pregnancy term typically lasts 9 out of 12 months
    - Note that this underestimates the number of pregnant women, because ASFR only considers the number of pregnancies resulting in live births (and excludes those resulting in premature termination or still births, for which no data is available by age). We also assume that there is no change in ASFR across districts, area types (rural/urban) and job-types.

<br>

$$ASFR = \frac{\text{Number of live births in a particular age-group}}{\text{Mid-year female population of the same age-group}} \times 1000$$

<br>
$$\text{Estimated number of pregnancies in an age-group} = \text{Female population of the same age-group} \times \frac{ASFR}{1000} \times \frac{9}{12}$$

<br>

<div align="center">
  <img src="https://drive.google.com/uc?export=view&id=1UU2gNfIopnCfzQzf2sMh5o_GlDf9n9X8" alt="My Image">
</div>

In [22]:
# Initialize the new column
aggregated_df.loc[:, 'Category'] = aggregated_df.loc[:, 'Abbreviated_Job_Type']

# Flag vulnerable workers based on age (less than 15 or greater than 69)
workers_age_conditions = (
    (aggregated_df['Age_Group'] == '5 to 9') |
    (aggregated_df['Age_Group'] == '10 to 14') |
    (aggregated_df['Age_Group'] == '70 to 79') |
    (aggregated_df['Age_Group'] == '80+')
)

workers_age_conditions &= (aggregated_df['Abbreviated_Job_Type'] != 'Non Workers')

# Apply the conditions
aggregated_df.loc[workers_age_conditions, 'Category'] = 'Job Type Agnostic Vulnerable'

<ipython-input-22-20d5ad8a653f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aggregated_df.loc[:, 'Category'] = aggregated_df.loc[:, 'Abbreviated_Job_Type']


In [23]:
# Use Age Specific Fertility Rate to compute Estimated Age Specific Fraction of
# Pregnant Women
ASFRs = {
    '15 to 19': 9.2,
    '20 to 24': 196.7,
    '25 to 29': 130.3,
    '30 to 34': 60.8,
    '35 to 39': 15.7,
    '40 to 49': (4.2 + 0.3)/2
}

age_specific_pregnant_fractions = {k: v/1000*9/12 for k,v in ASFRs.items()}
age_specific_pregnant_fractions

{'15 to 19': 0.0069,
 '20 to 24': 0.147525,
 '25 to 29': 0.097725,
 '30 to 34': 0.0456,
 '35 to 39': 0.011774999999999999,
 '40 to 49': 0.0016874999999999998}

In [24]:
# Initialise Pregnancy Status to False
aggregated_df['Pregnant'] = False

# Partition: workers dataset = (females with ages 15-49) + (others)
workers_female_fertile_conditions = (
    (aggregated_df['Age_Group'] == '15 to 19') |
    (aggregated_df['Age_Group'] == '20 to 24') |
    (aggregated_df['Age_Group'] == '25 to 29') |
    (aggregated_df['Age_Group'] == '30 to 34') |
    (aggregated_df['Age_Group'] == '35 to 39') |
    (aggregated_df['Age_Group'] == '40 to 49')
  )
workers_female_fertile_conditions &= (aggregated_df['Gender'] == 'Females')
workers_female_fertile_conditions &= \
 (aggregated_df['Abbreviated_Job_Type'] != 'Non Workers')

female_fertile_workers_df = \
  aggregated_df.loc[workers_female_fertile_conditions, :]
others_df = \
  aggregated_df.loc[~workers_female_fertile_conditions, :]

<ipython-input-24-f7183d12926b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aggregated_df['Pregnant'] = False


In [25]:
# Create an empty list to store all the rows (pregnant, non-pregnant, and others)
all_rows = []

# Iterate over the age-specific pregnant fractions
for age_group, pregnant_fraction in age_specific_pregnant_fractions.items():
    # Apply the mask for the current age group
    mask = (female_fertile_workers_df['Age_Group'] == age_group)
    df_subset = female_fertile_workers_df[mask].copy()  # Subset for the current age group

    # Estimate counts for pregnant women
    df_subset['Estimated_Pregnant'] = \
     (df_subset['Total_Count'] * pregnant_fraction).round()
    df_subset['Estimated_Main_Workers_Pregnant'] = \
     (df_subset['Main_Count'] * pregnant_fraction).round()
    df_subset['Estimated_Marginal_Workers_Pregnant'] = \
      df_subset['Estimated_Pregnant'] - \
      df_subset['Estimated_Main_Workers_Pregnant']

    # Create rows for pregnant / non-pregnant women
    pregnant_subset_df = df_subset[df_subset['Estimated_Pregnant'] > 0].copy() # if due to rounding, estimated pregnant is zero, no need for pregnant_subset_df
    non_pregnant_subset_df = df_subset.copy()

    # Update Worker Counts
    pregnant_subset_df['Total_Count'] = \
      pregnant_subset_df['Estimated_Pregnant']
    pregnant_subset_df['Main_Count'] = \
      pregnant_subset_df['Estimated_Main_Workers_Pregnant']
    pregnant_subset_df['Marginal_Count'] = \
      pregnant_subset_df['Estimated_Marginal_Workers_Pregnant']
    pregnant_subset_df['Category'] = 'Job Type Agnostic Vulnerable' # pregnant women are all vulnerable, irrespective job type
    pregnant_subset_df['Pregnant'] = True

    non_pregnant_subset_df['Total_Count'] = \
      non_pregnant_subset_df['Total_Count'] - \
      non_pregnant_subset_df['Estimated_Pregnant']
    non_pregnant_subset_df['Main_Count'] = \
      non_pregnant_subset_df['Main_Count'] - \
      non_pregnant_subset_df['Estimated_Main_Workers_Pregnant']
    non_pregnant_subset_df['Marginal_Count'] = \
      non_pregnant_subset_df['Marginal_Count'] - \
      non_pregnant_subset_df['Estimated_Marginal_Workers_Pregnant']

    # Append both pregnant and non-pregnant rows to the list
    all_rows.append(pregnant_subset_df)
    all_rows.append(non_pregnant_subset_df)

# Append the remaining rows (non-reproductive age groups and non-females)
all_rows.append(others_df)

# Concatenate all rows into the final DataFrame
updated_df = pd.concat(all_rows, ignore_index=True)

# Drop intermediate columns used for calculation
updated_df = updated_df.drop(columns=['Estimated_Pregnant',
                                      'Estimated_Main_Workers_Pregnant',
                                      'Estimated_Marginal_Workers_Pregnant'])
updated_df

,District_Code,Area_Name,Area_Type,Age_Group,Gender,Basic_Job_Type,Abbreviated_Job_Type,Main_Count,Marginal_Count,Total_Count,Category,Pregnant
0,090,District - North West,Rural,15 to 19,Females,"Arts, Entertainment and Recreation, Other Serv...",Arts and Other Sectors (Non-HHI),0.0,1.0,1.0,Job Type Agnostic Vulnerable,True
1,090,District - North West,Rural,15 to 19,Females,Manufacturing (Non-Household Industries),Manufacturing (Non-HHI),1.0,0.0,1.0,Job Type Agnostic Vulnerable,True
2,090,District - North West,Urban,15 to 19,Females,"Administrative and Support Service, Public Adm...",Administration,2.0,0.0,2.0,Job Type Agnostic Vulnerable,True
3,090,District - North West,Urban,15 to 19,Females,"Arts, Entertainment and Recreation, Other Serv...",Arts and Other Sectors (HHI),0.0,1.0,1.0,Job Type Agnostic Vulnerable,True
4,090,District - North West,Urban,15 to 19,Females,"Arts, Entertainment and Recreation, Other Serv...",Arts and Other Sectors (Non-HHI),15.0,3.0,18.0,Job Type Agnostic Vulnerable,True
...,...,...,...,...,...,...,...,...,...,...,...,...
10231,098,District - South,Urban,Age not stated,Males,Non Workers,Non Workers,0.0,0.0,953.0,Non Workers,False
10232,098,District - South,Urban,Age not stated,Males,"Plantation, Livestock, Forestry, Fishing, Hunt...",Other Food Production,3.0,0.0,3.0,Other Food Production,False
10233,098,District - South,Urban,Age not stated,Males,Transportation and Storage,Transportation & Storage,46.0,3.0,49.0,Transportation & Storage,False
10234,098,District - South,Urban,Age not stated,Males,Wholesale and Retail Trade (incl. Repair of mo...,Trade (HHI),0.0,0.0,0.0,Trade (HHI),False


# Reorder columns so that the Count Columns are shown towards the extreme right

In [26]:
# Reordered Column Names
reordered_columns = [updated_df.columns[i] \
                     for i in [0, 1, 2, 3, 4, 5, 6, 10, 11, 7, 8, 9]]

# Reorder the DataFrame columns
updated_df = updated_df[reordered_columns]

# Display the DataFrame to verify column order
updated_df.head()

,District_Code,Area_Name,Area_Type,Age_Group,Gender,Basic_Job_Type,Abbreviated_Job_Type,Category,Pregnant,Main_Count,Marginal_Count,Total_Count
0,090,District - North West,Rural,15 to 19,Females,"Arts, Entertainment and Recreation, Other Serv...",Arts and Other Sectors (Non-HHI),Job Type Agnostic Vulnerable,True,0.0,1.0,1.0
1,090,District - North West,Rural,15 to 19,Females,Manufacturing (Non-Household Industries),Manufacturing (Non-HHI),Job Type Agnostic Vulnerable,True,1.0,0.0,1.0
2,090,District - North West,Urban,15 to 19,Females,"Administrative and Support Service, Public Adm...",Administration,Job Type Agnostic Vulnerable,True,2.0,0.0,2.0
3,090,District - North West,Urban,15 to 19,Females,"Arts, Entertainment and Recreation, Other Serv...",Arts and Other Sectors (HHI),Job Type Agnostic Vulnerable,True,0.0,1.0,1.0
4,090,District - North West,Urban,15 to 19,Females,"Arts, Entertainment and Recreation, Other Serv...",Arts and Other Sectors (Non-HHI),Job Type Agnostic Vulnerable,True,15.0,3.0,18.0


In [27]:
# Create a new worksheet
new_worksheet = sheet.add_worksheet(title="Processed Data",
                                    rows=f"{updated_df.shape[0]}",
                                    cols=f"{updated_df.shape[1]}")

# Write the long format dataframe back to Google Sheets
set_with_dataframe(new_worksheet, updated_df)